## Maximum Elder facilities Covering Location Problem  solved by Genetic Algorithm (GA)

In [ ]:
from Algorithm.GA_BSCLP import ImprovedGeneticAlgorithm as GeneticAlgorithm
import random
import numpy as np
from matplotlib import pyplot as plt

## Load the real-world datasets

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import time
import seaborn as sns
from matplotlib.patches import Circle

### Demand Population Distribution

In [ ]:
%%time
ls = gpd.read_file(r"D:\Academic\Task116_ApplyScience\allover\BSLCP\data\电网\powertower_tianhe_a\HaizhuPowertower_3857.shp")
ls['POINT_X'] = ls.geometry.x
ls['POINT_Y'] = ls.geometry.y
ls.head(3)

In [ ]:
def normalize_to_1_10(data):
    """
    将一组数据归一化到1-10
    """
    min_val = min(data)
    max_val = max(data)
    normalized_data = [(x - min_val) / (max_val - min_val) * 9 + 1 for x in data]
    return normalized_data
# times -1 for maximizing the sum
ls['speed_pct_freeflow_rev'] = ls.a
ls['speed_pct_freeflow_rev_norm'] = normalize_to_1_10(ls['speed_pct_freeflow_rev'])

total_pop = sum(ls['speed_pct_freeflow_rev'])
total_pop_norm = sum(ls['speed_pct_freeflow_rev_norm'])
print("The number of records is ", len(ls))
print("The total speed unit are ", total_pop)
print("The total norm speed unit are ", total_pop_norm)

In [ ]:
sitedf = pd.read_csv(r"D:\Academic\Task116_ApplyScience\allover\BSLCP\data\电网\powertower_tianhe_a\HaizhuPowertower.csv")
# sites = np.array(sitedf[['NORM_X', 'NORM_Y']], dtype=np.float64)
print("The number of billboards in Seattle area is ", len(sitedf))
sitedf.head(3)

## Normalization

In [ ]:
def Normalization(x, y):
    max_x, max_y = np.max(x), np.max(y)
    min_x, min_y = np.min(x), np.min(y)
    S_x = (max_x-min_x)
    S_y = (max_y-min_y)
    S = max(S_x, S_y)
    new_x, new_y = (x-min_x)/S, (y-min_y)/S
    data_xy = np.vstack((new_x, new_y))
    Data = data_xy.T
    return new_x, new_y, S

In [ ]:
ls_X = np.array(ls['POINT_X'])
ls_Y = np.array(ls['POINT_Y'])
bbs_X = np.array(sitedf['POINT_X'])
bbs_Y = np.array(sitedf['POINT_Y'])
X = np.concatenate([ls_X, bbs_X])
Y = np.concatenate([ls_Y, bbs_Y])
NORM_X, NORM_Y, S = Normalization(X, Y)
ls['NORM_X'] = NORM_X[:len(ls)]
ls['NORM_Y'] = NORM_Y[:len(ls)]
sitedf['NORM_X'] = NORM_X[len(ls):]
sitedf['NORM_Y'] = NORM_Y[len(ls):]

In [ ]:
def generate_candidate_sites(sites, M=100, heuristic = None):
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        sites: a Pandas DataFrame with X, Y and other characteristic
        M: the number of candidate sites to generate
        heuristic:
    Return:
        sites: a Numpy array with shape of (M,2)
    '''

    if M is not None:
        if M > len(sites):
            M = None
    if heuristic is None or heuristic == '':
        if M is None:
            return sites
        np.random.seed(0)
        index = np.random.choice(len(sites), M)
        return sites.iloc[index]
    elif heuristic == 'coverage':
        sites = sites.sort_values(by='pop_covered_2km', ascending=False).reset_index()
        if M is None:
            return sites
        return sites.iloc[:M]
    elif heuristic == 'coverage_e':
        sites = sites.sort_values(by='pop_covered_2km_exclusive', ascending=False).reset_index()
        if M is None:
            return sites
        return sites.iloc[:M]
    elif heuristic == 'impression':
        sites = sites.sort_values(by='weeklyImpr', ascending=False).reset_index()
        if M is None:
            return sites
        return sites.iloc[:M]
    elif heuristic == 'impression_e':
        sites = sites.sort_values(by='weeklyImpr_2km_exclusive', ascending=False).reset_index()
        if M is None:
            return sites
        return sites.iloc[:M]

In [ ]:
bbs_ = generate_candidate_sites(sitedf, M=None, heuristic="")

In [ ]:
users = np.array(ls[['NORM_X', 'NORM_Y']])
facilities = np.array(bbs_[['NORM_X', 'NORM_Y']])
demand = np.array(ls['speed_pct_freeflow_rev'])
p = 25
real_radius = 1000
radius = real_radius/S
print(S)

dist = np.sum((facilities[:, np.newaxis, :] - users[np.newaxis, :, :]) ** 2, axis=-1) ** 0.5
A = np.sum((facilities[:, np.newaxis, :] - users[np.newaxis, :, :]) ** 2, axis=-1) ** 0.5
mask1 = A <= radius
A[mask1] = 1
A[~mask1] = 0

In [ ]:
w = 0.8
start_time = time.time()
distance_matrix = np.sqrt(np.sum((facilities[:, np.newaxis, :] - users[np.newaxis, :, :]) ** 2, axis=-1))

genetic = GeneticAlgorithm(n=len(ls), m=len(bbs_), p=p, cost_matrix=distance_matrix, r=radius,
                               demand=demand, w=w, elite_num=2)
genetic.optimize()

    # 最终的目标函数值是 top_chromosome.fitness 的相反数
obj = -genetic.top_chromosome.fitness
centers = genetic.top_chromosome.content
time = genetic.time


print("The Set of centers are:", centers)
print("Objective value (no penalty):", round(obj, 2))
print(f"time: {time} seconds")
# # 1. 初始化用于存储每次运行结果的列表
# total_objectives = []
# total_coverages = []
# total_times = []
#
# # To store the details of the best run
# best_overall_objective = -np.inf
# best_overall_solution = None
# best_overall_coverage = 0
#
# # Set the number of runs
# num_runs = 1000
#
# print(f"Starting {num_runs} runs of the Genetic Algorithm...")
#
# # 2. Loop through the runs
# for i in range(num_runs):
#     # Print progress
#     print(f"--- Running {i+1}/{num_runs} ---")
#
#
#     # --- Core part of your original code ---
#     w1 = 0.6
#     w2 = 0.4
#     da = 1000 / S
#     db = 2000 / S
#     D_max = 2000 / S
#
#     start_time = time.time()
#
#     selected_indices = np.random.choice(len(facilities), 100, replace=False)
#     selected_indices.sort()
#     subset_facilities = facilities[selected_indices]
#     subset_dist = dist[selected_indices, :]
#     subset_A = A[selected_indices, :]
#
#
#     genetic = GeneticAlgorithmVectorized(n=len(ls), m=len(subset_facilities), p=p,
#         users=users, facilities=subset_facilities,
#         demand=demand, w1=w1, w2=w2,
#         da=da, db=db, D_max=D_max,
#         A=subset_A, dist=subset_dist)
#
#     centers, obj, coverage = genetic.optimize()
#
#     end_time = time.time()
#     execution_time = end_time - start_time
#     # --- End of core part ---
#
#     # 3. Add results from the current run to lists
#     total_objectives.append(obj)
#     total_coverages.append(coverage)
#     total_times.append(execution_time)
#
#     # 4. Check if this run is the best one so far
#     if obj > best_overall_objective:
#         best_overall_objective = obj
#         best_overall_solution = centers
#         best_overall_coverage = coverage
#         print(f"!!! New best solution found. Objective: {round(obj, 2)} !!!")
#
#
# print(f"\nAll {num_runs} runs completed!")
#
# # 5. Calculate average results
# avg_objective = np.mean(total_objectives)
# avg_coverage = np.mean(total_coverages)
# avg_time = np.mean(total_times)
#
# # 6. Output the final average results and the best sequence
# print("\n--- Average Results ---")
# print(f"Average Objective value: {round(avg_objective, 2)}")
# print("Average Coverage rate: {:.2%}".format(avg_coverage))
# print(f"Average time: {avg_time:.4f} seconds")
#
# print("\n--- Best Overall Result ---")
# print(f"Maximum Objective value found: {round(best_overall_objective, 2)}")
# print(f"Coverage rate for the best run: {best_overall_coverage:.2%}")
# print(f"The sequence that produced the maximum value is: \n{best_overall_solution}")

In [ ]:

def plot_result(ls, opt_sites, radius):
    '''
    Plot the result
    Input:
        points: input points, Numpy array in shape of [N,2]
        opt_sites: locations K optimal sites, Pandas DataFrame
        radius: the radius of circle
    '''
    fig, ax = plt.subplots(figsize=(20, 12))
    # 创建一个自定义的颜色映射
    cmap = plt.cm.Blues
    new_cmap = colors.ListedColormap(cmap(np.linspace(0.15, 1, 256)))
    ls.plot(ax=ax, column=ls['speed_pct_freeflow_rev'], k=5, markersize=5, cmap=new_cmap, label='over-65 Population')
    legend_plot_flag = {'current': False, 'selected': False}

    for cnt, site in opt_sites.iterrows():
        if 'current' in opt_sites.columns and site['current'] == True:  #ncurrent
            if legend_plot_flag['current'] == False:
                ax.scatter(site['POINT_X'], site['POINT_Y'], c='red', marker='+', s=10, label='Current Billboards')
                circlelabel = 'Service Range for Current Billboards'
                legend_plot_flag['current'] = True
            ax.scatter(site['POINT_X'], site['POINT_Y'], c='red', marker='+', s=100)
            #             circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='red', fill=False, lw=2, label = circlelabel)
            circle = plt.Circle(site[['POINT_X', 'POINT_Y']], radius, color='red', fill=False, lw=2)

            ax.add_artist(circle)
        else:
            if legend_plot_flag['selected'] == False:
                ax.scatter(site['POINT_X'], site['POINT_Y'], c='C1', marker='+', s=10,
                           label='Optimized Selected Elder Facilities')
                circlelabel = 'Service Range for Optimized Selected Billboards'
                legend_plot_flag['selected'] = True
            ax.scatter(site['POINT_X'], site['POINT_Y'], c='C1', marker='+', s=100)
            #             circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='C1', fill=False, lw=2, label = circlelabel)
            circle = plt.Circle(site[['POINT_X', 'POINT_Y']], radius/2, color='C1', fill=False, lw=2)
            ax.add_artist(circle)

    ax.axis('scaled')
    ax.tick_params(axis='both', left=False, top=False, right=False,
                   bottom=False, labelleft=False, labeltop=False,
                   labelright=False, labelbottom=False)
    title = 'Selected ' + str(len(opt_sites)) + ' Sites that Serve ' + str(radius) + ' m(GA)'
    ax.set_title(title, fontsize=20)
    render_scale_bar(ax=ax, x=0.05, y=0.05)
    render_north_arrow(ax=ax, x=0.95, y=0.95, size=0.01, ratio=0.7)
    ax.legend(loc='lower right', markerscale=2)
    return ax

In [ ]:
def render_scale_bar(ax, x, y, segments=2, height=0.01, seg_length=2000, unit='m', linewidth=1.):
    unit_scale_factor = {
        'm': 1,
        'km': 1000,
        'meters': 1,
        'kilometers': 1000,
        'miles': 1609.34,
        'mi': 1609.34,
        'ft': 0.3,
        }
    x_lim = ax.get_xlim()
    y_lim = ax.get_ylim()

    # how much percent does one unit takes on the x axis
    x_per_unit = 1. / (x_lim[1] - x_lim[0])
    y_per_unit = 1. / (y_lim[1] - y_lim[0])

    # base for ticks (0, 1)
    x_base = [x + seg_length * unit_scale_factor[unit] * x_per_unit * i for i in range(0, segments + 1)]
    ax.axhline(y_lim[0] + y / y_per_unit, x_base[0], x_base[-1], c='black')
    y_base = [y, y + height]
    for i in range(segments + 1):
        ax.axvline(x_lim[0] + x_base[i] / x_per_unit, y, y + height, c='black')
        xy = (x_lim[0] + x_base[i] / x_per_unit, y_lim[0] + (y - 0.015) / y_per_unit)  # data_coords
        ax.text(xy[0], xy[1], s='{}'.format(int(seg_length * i)), horizontalalignment='center', verticalalignment='center')
    ax.text(x_lim[0] + (x_base[-1] + 0.02) / x_per_unit, y_lim[0] + (y - 0.015) / y_per_unit,
            s=unit, horizontalalignment='left',
            verticalalignment='center')
    ax.grid(False)
def render_north_arrow(ax, x, y, size, ratio = 1):
    path = [(0, 1), (-ratio, -1), (0, -0.5), (ratio, -1), (0, 1)]
    path = [(i[0] * size + x, i[1] * size + y) for i in path]
    arrow = plt.Polygon(path, color='black', transform=ax.transAxes)
    ax.add_patch(arrow)
    ax.text(x, y-size*2, s = 'N', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

In [ ]:
best_overall_solution = [156, 79, 92, 220, 256, 167, 150, 17, 108, 185, 288, 27, 12, 114, 210, 125, 127, 22, 89, 205, 68, 206, 62, 174, 101]
opt_sites = bbs_.iloc[best_overall_solution]
plot_result(ls,opt_sites,1000)

In [ ]:
best_overall_solution_improved_GA = [156, 79, 92, 220, 256, 167, 150, 17, 108, 185, 288, 27, 12, 114, 210, 125, 127, 22, 89, 205, 68, 206, 62, 174, 101]
opt_sites = bbs_.iloc[best_overall_solution_improved_GA]
plot_result(ls,opt_sites,1000)

In [ ]:
# 收敛曲线：记录每代最优目标函数值并绘图
import time as pytime

class GeneticAlgorithmWithHistory(GeneticAlgorithm):
    """
    在优化过程中记录 best-so-far 目标函数值序列（收敛曲线）。
    说明：GA 基类最小化 fitness (= -objective)，故记录的 objective = -fitness。
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.history = []  # 每代的 best-so-far 目标函数值（越大越好）

    def optimize(self):
        start_time = pytime.time()
        chromosomes = self.initial_random_population()
        # 记录初始解对应的目标函数值
        self.history = [-self.top_chromosome.fitness]
        while self.current_iteration < self.iterations:
            for_reproduction = self.selection(chromosomes)
            chromosomes = self.create_generation(for_reproduction)
            self.current_iteration += 1
            chromosome_with_min_fitness = min(chromosomes, key=lambda chromo: chromo.fitness)
            if chromosome_with_min_fitness.fitness < self.top_chromosome.fitness:
                self.top_chromosome = chromosome_with_min_fitness
            # 记录迭代后的 best-so-far 目标函数值
            self.history.append(-self.top_chromosome.fitness)
        end_time = pytime.time()
        self.time = end_time - start_time
        hours, rem = divmod(end_time - start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print("\nFinal top solution: %s" % self.top_chromosome)
        print('Time: {:0>2}:{:0>2}:{:05.4f}'.format(int(hours), int(minutes), seconds))
        return self.top_chromosome.content, -self.top_chromosome.fitness, self.history


def plot_convergence_curve(distance_matrix, p, radius, demand, w=0.8,
                           iterations=None, generation_size=None, reproduction_size=None,
                           mutation_prob=None, elite_num=2, figsize=(8,4)):
    """
    绘制 GA 收敛曲线：随着代数迭代 best-so-far 目标函数值的变化。
    参数：
      - distance_matrix: np.ndarray [M,N] 设施-需求的距离矩阵
      - p: 选择设施个数
      - radius: 覆盖半径（与 distance_matrix 同尺度）
      - demand: 长度 N 的需求数组
      - w: 目标函数权重（主/备覆盖）
      - iterations/generation_size/reproduction_size/mutation_prob: 可选地覆盖 GA 超参
      - elite_num: 精英保留个数（Improved GA 专用）
    返回：history（每代 best-so-far 目标函数值）
    """
    n_users = distance_matrix.shape[1]
    m_facilities = distance_matrix.shape[0]
    ga = GeneticAlgorithmWithHistory(n=n_users, m=m_facilities, p=p,
                                     cost_matrix=distance_matrix, r=radius,
                                     demand=np.array(demand, dtype=float), w=w, elite_num=elite_num)
    if iterations is not None:
        ga.iterations = int(iterations)
    if generation_size is not None:
        ga.generation_size = int(generation_size)
    if reproduction_size is not None:
        ga.reproduction_size = int(reproduction_size)
    if mutation_prob is not None:
        ga.mutation_prob = float(mutation_prob)

    ga.optimize()

    # 绘图
    plt.figure(figsize=figsize)
    plt.plot(range(len(ga.history)), ga.history, color='C0', marker='o', markersize=3, linewidth=1,
             label='Best-so-far objective')
    plt.xlabel('Generation')
    plt.ylabel('Objective value')
    plt.title('GA Convergence Curve (w={:.2f}, p={})'.format(w, p))
    plt.grid(alpha=0.3)
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.show()

    return ga.history

# 使用示例（运行前确保已定义 distance_matrix, p, radius, demand, w）：
hist = plot_convergence_curve(distance_matrix, p, radius, demand, w=w, iterations=200, elite_num=2)



In [ ]:
# 直接绘制当前设置下（distance_matrix, p, radius, demand, w）的收敛曲线
# 运行前请先执行上文，确保已定义 distance_matrix, p, radius, demand, w
hist = plot_convergence_curve(distance_matrix, p, radius, demand, w=w, iterations=200, elite_num=2)
print('History length:', len(hist))
print('Best objective:', max(hist) if len(hist)>0 else None)


In [ ]:
# 多超参数设置的收敛曲线对比（同一张图）
# 运行前请确保 distance_matrix, p, radius, demand, w 已定义

import numpy as np, random
import pandas as pd
import matplotlib.pyplot as plt

# 固定随机种子，便于对比复现（如不需要可注释掉）
random.seed(0); np.random.seed(0)

n_users = distance_matrix.shape[1]
m_facilities = distance_matrix.shape[0]

# 超参数组：可按需增删
param_sets = [
    {"label": "gen=50, repr=20, mut=0.10, elite=2", "iterations": 200, "generation_size": 50, "reproduction_size": 20, "mutation_prob": 0.10, "elite_num": 2},
    {"label": "gen=50, repr=20, mut=0.20, elite=2", "iterations": 200, "generation_size": 50, "reproduction_size": 20, "mutation_prob": 0.20, "elite_num": 2},
    {"label": "gen=80, repr=30, mut=0.10, elite=2", "iterations": 200, "generation_size": 80, "reproduction_size": 30, "mutation_prob": 0.10, "elite_num": 2},
    {"label": "gen=50, repr=20, mut=0.10, elite=0", "iterations": 200, "generation_size": 50, "reproduction_size": 20, "mutation_prob": 0.10, "elite_num": 0},
]

histories = []
summary_rows = []

for ps in param_sets:
    ga = GeneticAlgorithmWithHistory(n=n_users, m=m_facilities, p=p,
                                     cost_matrix=distance_matrix, r=radius,
                                     demand=np.array(demand, dtype=float), w=w, elite_num=ps["elite_num"]) 
    ga.iterations = int(ps["iterations"]) 
    ga.generation_size = int(ps["generation_size"]) 
    ga.reproduction_size = int(ps["reproduction_size"]) 
    ga.mutation_prob = float(ps["mutation_prob"]) 

    ga.optimize()

    hist = ga.history
    histories.append((ps["label"], hist))
    summary_rows.append({
        "label": ps["label"],
        "iterations": len(hist) - 1,
        "best_objective": float(max(hist)) if len(hist) > 0 else np.nan,
        "final_objective": float(hist[-1]) if len(hist) > 0 else np.nan,
        "generation_size": ps["generation_size"],
        "reproduction_size": ps["reproduction_size"],
        "mutation_prob": ps["mutation_prob"],
        "elite_num": ps["elite_num"],
    })

# 绘图：同一坐标系
plt.figure(figsize=(12, 6))
for label, hist in histories:
    plt.plot(range(len(hist)), hist, linewidth=1.6, label=label)
plt.xlabel("Generation")
plt.ylabel("Objective value")
plt.title("GA convergence under different hyperparameters")
plt.grid(alpha=0.3)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

# 汇总表
df_compare = pd.DataFrame(summary_rows)
display(df_compare.sort_values("best_objective", ascending=False).reset_index(drop=True))



In [ ]:
# Print hardware/software environment versions (for paper reproducibility)
import platform, sys, importlib
import pandas as pd

# Gurobi version (robust)
try:
    import gurobipy as gp
    try:
        vt = gp.gurobi.version()
        gurobi_version = '.'.join(map(str, vt))
    except Exception:
        gurobi_version = getattr(gp, '__version__', 'Unknown')
except Exception:
    gurobi_version = 'Not installed'

# Generic version getter
def get_version(module_name: str, attr: str = '__version__') -> str:
    try:
        m = importlib.import_module(module_name)
        v = getattr(m, attr, None)
        if v is None:
            # some packages expose version differently
            if module_name == 'matplotlib':
                import matplotlib
                v = matplotlib.__version__
            else:
                v = 'Unknown'
        return str(v)
    except Exception:
        return 'Not installed'

rows = [
    ('OS', f"{platform.system()} {platform.release()} ({platform.version()})"),
    ('Machine', platform.machine()),
    ('Python', platform.python_version()),
    ('Gurobi', gurobi_version),
    ('NumPy', get_version('numpy')),
    ('Pandas', get_version('pandas')),
    ('Matplotlib', get_version('matplotlib')),
    ('Seaborn', get_version('seaborn')),
    ('scikit-learn', get_version('sklearn')),
    ('GeoPandas', get_version('geopandas')),
    ('Shapely', get_version('shapely')),
    ('Fiona', get_version('fiona')),
    ('PyProj', get_version('pyproj')),
    ('QGIS (qgis)', get_version('qgis')),
    ('ArcGIS (arcpy)', get_version('arcpy')),
]

df_env = pd.DataFrame(rows, columns=['Component', 'Version'])
display(df_env)

# Optional: also print plain text for easy copy-paste
for k, v in rows:
    print(f"{k}: {v}")

